# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [4]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [5]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

\Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [6]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [7]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [8]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [9]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [10]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [11]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [12]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [13]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [14]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [15]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [16]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 3.088469709996792),
 (1, 3.088469709996792),
 (2, 3.088469709996792),
 (3, 3.088469709996792),
 (4, 3.088469709996792)]

## Inverted index

In [17]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [18]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [19]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [20]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2)]),
 (1, [('banana', 2), ('is', 18), ('it', 18), ('what', 10)])]

## TeraSort

In [21]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.04110726414264221),
   (None, 0.08199859623376693),
   (None, 0.10795806769865379),
   (None, 0.15479002823736965),
   (None, 0.16191493220352604),
   (None, 0.18324236317350195),
   (None, 0.23954558477391796),
   (None, 0.28253568628665027),
   (None, 0.2973181196917315),
   (None, 0.33434365709795466),
   (None, 0.362464542069426),
   (None, 0.3674763600374078),
   (None, 0.41877678853436295),
   (None, 0.4267207523176819),
   (None, 0.4425494766302921),
   (None, 0.4797048621127654),
   (None, 0.49626849583926147)]),
 (1,
  [(None, 0.5122822510458037),
   (None, 0.5445429685598041),
   (None, 0.556589445529854),
   (None, 0.5952689737282907),
   (None, 0.6959990705957939),
   (None, 0.7565995304129246),
   (None, 0.7677639854486074),
   (None, 0.8596518685595436),
   (None, 0.9052127967169827),
   (None, 0.9465766184392347),
   (None, 0.9600522479299635),
   (None, 0.9695750094186412),
   (None, 0.9741058641210396)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [23]:
import random

row = [random.randint(-100, 100) for _ in range(20)]

def RECORDREADER():
  yield (0, row)

def MAP(_, row):
  yield (0, max(row))

def REDUCE(_, row):
  yield max(row)

max_output = MapReduce(RECORDREADER, MAP, REDUCE)
list(max_output)

[96]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [26]:
row = [random.randint(-100, 100) for _ in range(20)]

def RECORDREADER():
  yield (0, row)

def MAP(_, row):
  for number in row:
    yield (0, number)

def REDUCE(_, nums):
  count = 0
  sum = 0
  for num in nums:
    count += 1
    sum += num
  yield sum / count

sum_output = MapReduce(RECORDREADER, MAP, REDUCE)
list(sum_output)

[2.55]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [29]:
d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId: str, body: str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word: str, docId: Iterator[str]):
  yield (word, sorted(docId))


def groupbykey(iterable):
  t = []
  index = -1
  prev_k2 = None
  for (k2, v2) in sorted(iterable, key=lambda x: x[0]):
    if k2 != prev_k2:
      index += 1
      prev_k2 = k2
      t.append(tuple([k2, []]))
    t[index][1].append(v2)
  return t

sotred_output = MapReduce(RECORDREADER, MAP, REDUCE)
list(sotred_output)

[('a', ['2']),
 ('banana', ['2']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('what', ['0', '1'])]

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [30]:
from functools import reduce
row = [random.randint(0, 5) for _ in range(20)]

maps = 2
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(index, split):
    for number in split:
      yield (index, number)

  split_size = int(np.ceil(len(row) / maps))

  for index, i in enumerate(range(0, len(row), split_size)):
    yield RECORDREADER(index, row[i:i+split_size])

def MAP(index, number):
  yield (index, number)

def REDUCE(_, numbers):
  yield list(set(numbers))


partitioned_set_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER= None, PARTITIONER= PARTITIONER)
partitioned_set_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_set_output]
partitioned_set_output

20 key-value pairs were sent over a network.


[(0, [[0, 1, 2, 3, 4]]), (1, [[0, 1, 3, 4, 5]])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [31]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def C(row):
  return row.social_contacts >= 250

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

def MAP(_, t):
  if (C(t)):
    yield (t, t)

def REDUCE(t, ts):
  for it in ts:
    yield (it, it)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(map(lambda x: x[0], output))
output

[User(id=2, age=25, social_contacts=500, gender='female'),
 User(id=3, age=33, social_contacts=800, gender='female')]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [32]:
class Projection(NamedTuple):
  gender: str
  social_contacts: int

def S(row: NamedTuple):
  return Projection(gender = row.gender, social_contacts=row.social_contacts)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

def MAP(_, t):
  t_ = S(t)
  yield (t_, t_)

def REDUCE(t, ts):
  yield (t, t)

projection_output = MapReduce(RECORDREADER, MAP, REDUCE)
list(map(lambda x: x[0], projection_output))

[Projection(gender='female', social_contacts=240),
 Projection(gender='female', social_contacts=500),
 Projection(gender='female', social_contacts=800),
 Projection(gender='male', social_contacts=20)]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [35]:
input_collection_1 = [
    User(id=0, age=20, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection_2 = [
    User(id=0, age=20, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=4, age=55, gender='male', social_contacts=120)
]


def RECORDREADER():
  for it in input_collection_1:
    yield (1, it)
  for it in input_collection_2:
    yield (2, it)

def MAP(_, t):
  yield (t, t)

def REDUCE(t, ts):
  yield (t, t)

union_output = MapReduce(RECORDREADER, MAP, REDUCE)
list(map(lambda x: x[0], union_output))

[User(id=0, age=20, social_contacts=20, gender='male'),
 User(id=1, age=25, social_contacts=240, gender='female'),
 User(id=2, age=25, social_contacts=500, gender='female'),
 User(id=3, age=33, social_contacts=800, gender='female'),
 User(id=4, age=55, social_contacts=120, gender='male')]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [38]:
input_collection_1 = [
    User(id=0, age=20, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection_2 = [
    User(id=0, age=20, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=4, age=55, gender='male', social_contacts=120)
]


def RECORDREADER():
  for it in input_collection_1:
    yield (1, it)
  for it in input_collection_2:
    yield (2, it)

def MAP(_, t):
  yield (t, t)

def REDUCE(t, ts):
  if len(ts) == 2:
    yield (t, t)

intersection_output = MapReduce(RECORDREADER, MAP, REDUCE)
list(map(lambda x: x[0], intersection_output))

[User(id=0, age=20, social_contacts=20, gender='male'),
 User(id=1, age=25, social_contacts=240, gender='female')]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [40]:
input_collection_1 = [
    User(id=0, age=20, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection_2 = [
    User(id=0, age=20, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=4, age=55, gender='male', social_contacts=120)
]

def RECORDREADER():
  for it in input_collection_1:
    yield (1, it)
  for it in input_collection_2:
    yield (2, it)

def MAP(r, t):
  yield(t, r)

def REDUCE(t, rs):
  if len(rs) == 1 and rs[0] == 1:
    yield (t, t)

dif_output = MapReduce(RECORDREADER, MAP, REDUCE)
list(map(lambda x: x[0], dif_output))

[User(id=2, age=25, social_contacts=500, gender='female'),
 User(id=3, age=33, social_contacts=800, gender='female')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [41]:
class Gender(NamedTuple):
  gender: str
  power: int

class UserGender(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
  power: int

input_collection_1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

input_collection_2 = [
    Gender(gender='male', power=2),
    Gender(gender='female', power=1)
]

def K(row):
  return row.gender

def create_join(r, s, gender):
  return UserGender(id = r.id, age = r.age, social_contacts=r.social_contacts, gender = gender, power = s.power)

def RECORDREADER():
  for it in input_collection_1:
    yield (0, it)
  for it in input_collection_2:
    yield (1, it)

def MAP(r, t):
  yield (K(t), (r, t))

def REDUCE(k, rts):
  R = list(filter(lambda x: x[0] == 0, rts))
  S = list(filter(lambda x: x[0] == 1, rts))

  for r in R:
    for s in S:
      yield create_join(r[1], s[1], k)

join_output = MapReduce(RECORDREADER, MAP, REDUCE)
list(join_output)

[UserGender(id=1, age=25, social_contacts=240, gender='female', power=1),
 UserGender(id=2, age=25, social_contacts=500, gender='female', power=1),
 UserGender(id=3, age=33, social_contacts=800, gender='female', power=1),
 UserGender(id=0, age=55, social_contacts=20, gender='male', power=2)]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [42]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def G(row):
  return row.gender

def A(row):
  return row.social_contacts


def agregate(x):
  return sum(x) / len(x)

def RECORDREADER():
  for it in input_collection:
    yield (None, it)

def MAP(_, t):
  yield (G(t), A(t))

def REDUCE(g, a):
  yield (g, agregate(a))

agreagte_output = MapReduce(RECORDREADER, MAP, REDUCE)
list(agreagte_output)

[('female', 513.3333333333334), ('male', 20.0)]

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [46]:
mat = np.array([(np.ones([4])*i) for i in range(10)])
vec = np.array([1, 2, 3, 4])

maps = 3
reducers = 1

def INPUTFORMAT():
  global maps

  def RECORDREADER(index, split):
    for i in range(split.shape[0]):
      for j in range(split.shape[1]):
        yield (index + i, (split[i, j], vec[j]))

  split_size = int(np.ceil(mat.shape[0] / maps))
  for i in range(0, mat.shape[0], split_size):
    yield RECORDREADER(i, mat[i:i + split_size, :])

def MAP(coordinates, value):
  i = coordinates
  mat_v, vec_v = value
  yield (i, mat_v * vec_v)

def REDUCE(row, values):
  yield (row, sum(values))


matrix_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER = None, PARTITIONER=PARTITIONER)
[(partition_id, list(partition)) for (partition_id, partition) in matrix_output]

40 key-value pairs were sent over a network.


[(0,
  [(0, 0.0),
   (1, 10.0),
   (2, 20.0),
   (3, 30.0),
   (4, 40.0),
   (5, 50.0),
   (6, 60.0),
   (7, 70.0),
   (8, 80.0),
   (9, 90.0)])]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [47]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [50]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  yield (key, sum(values))

Проверьте своё решение

In [52]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [53]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [54]:
def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      for k in range(big_mat.shape[1]):
        yield ((i,k), (1, small_mat[i,j]))

  for i in range(small_mat.shape[0]):
    for j in range(big_mat.shape[0]):
      for k in range(big_mat.shape[1]):
        yield ((i,k), (2, big_mat[j,k]))


def MAP(k1, v1):
  yield (k1, v1)


def REDUCE(key, values):
  small_m = np.array([val for idx, val in values if idx == 1])
  big_m =  np.array([val for idx, val in values if idx == 2])
  yield (key, small_m @ big_m)

solution = MapReduce(RECORDREADER, MAP, REDUCE)
np.allclose(small_mat @ big_mat, asmatrix(solution))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [57]:
maps = 2
reducers = 1

def INPUTFORMAT():
  global maps

  def RECORDREADER_SMALL(index, split):
    for i in range(split.shape[0]):
      for j in range(split.shape[1]):
        for k in range(big_mat.shape[1]):
          yield ((i+index,k), (1, split[i,j]))

  split_size =  int(np.ceil(small_mat.shape[0]/maps))
  for i in range(0, small_mat.shape[0], split_size):
    yield RECORDREADER_SMALL(i, small_mat[i:i+split_size,:])

  def RECORDREADER_BIG(idx, split):
    for i in range(small_mat.shape[0]):
      for j in range(split.shape[0]):
        for k in range(split.shape[1]):
          yield ((i,k+idx), (2, split[j,k]))

  split_size =  int(np.ceil(big_mat.shape[0]/maps))
  for k in range(0, big_mat.shape[1], split_size):
    yield RECORDREADER_BIG(k, big_mat[:, k:k+split_size])

def MAP(k1, v1):
  yield (k1, v1)

def REDUCE(key, values):
  small_m = np.array([val for idx, val in values if idx == 1])
  big_m =  np.array([val for idx, val in values if idx == 2])
  yield (key, small_m @ big_m)

partitioned_solution = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_solution = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_solution]


def asmatrix_distributed(reduce_output):
    merge_output = []

    for _, values in reduce_output:
        for value in values:
            merge_output.append(value)

    combine_output = {}
    for key, value in merge_output:
        if key in combine_output:
            combine_output[key] += value
        else:
            combine_output[key] = value

    return asmatrix((key, val) for key, val in combine_output.items())

np.allclose(small_mat @ big_mat, asmatrix_distributed(partitioned_solution))

480 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [63]:
maps = 2
reducers = 1

def randomize_indixes(matrix):
    index = [(row, col) for row in range(matrix.shape[0]) for col in range(matrix.shape[1])]
    random.shuffle(index)
    return index

def INPUTFORMAT():
    def RECORDREADER_SMALL(submatrix, index):
        row, col = index
        for i in range(submatrix.shape[0]):
            for j in range(submatrix.shape[1]):
              for k in range(big_mat.shape[1]):
                  yield ((i + row, k), (0, col, submatrix[i, j]))

    small_idx = randomize_indixes(small_mat)
    for (row, col) in small_idx:
        yield RECORDREADER_SMALL(np.array([[small_mat[row, col]]]), (row, col))

    def RECORDREADER_BIG(submatrix, idx):
        row, col = idx
        for i in range(small_mat.shape[0]):
            for j in range(submatrix.shape[0]):
                for k in range(submatrix.shape[1]):
                    yield ((i, k + col), (1, row, submatrix[j, k]))

    big_idx = randomize_indixes(big_mat)
    for (row, col) in big_idx:
        yield RECORDREADER_BIG(np.array([[big_mat[row, col]]]), (row, col))

def MAP(k1, v1):
    yield (k1, v1)

def REDUCE(key, values):
    sorted_m = sorted(values, key=lambda x: x[1])
    small_m = np.array([val for index, col, val in sorted_m if index == 0])
    big_m = np.array([val for index, row, val in sorted_m if index == 1])
    yield (key, small_m @ big_m)


general_solution = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
general_solution = [(partition_id, list(partition)) for (partition_id, partition) in general_solution]

np.allclose(small_mat @ big_mat, asmatrix_distributed(general_solution))

480 key-value pairs were sent over a network.


True